<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Pre-trained-LM-selection-and-training/Multi_label_Language_Model_training_Ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-label dataset 

In [ ]:
import pandas as pd

multi_class = pd.read_csv('/content/exp5_multi_class_imbalance_handled.csv', index_col=0) 
multi_label = pd.read_csv('/content/multi_label_explicit_corrected.csv', index_col=0)

In [ ]:
multi_class.bias_type.value_counts()

anti-stereotype    5737
unrelated          4229
Ethnicity          2613
religion           1953
profession         1557
gender             1012
Name: bias_type, dtype: int64

In [ ]:
multi_label.Type.value_counts()

Anti-stereotype    5443
stereotype         5443
Name: Type, dtype: int64

In [ ]:
multi_label.bias_type.value_counts()

Ethnicity     5226
profession    3112
gender        2024
religion       524
Name: bias_type, dtype: int64

Extract religion bias and merge into multi-label

In [ ]:
religion = multi_class.loc[multi_class['bias_type'] == 'religion']

In [ ]:
religion.reset_index(drop = True,inplace=True)

In [ ]:
religion = religion.assign(Type = "stereotype")

In [ ]:
religion1 = multi_label[multi_label.bias_type != 'religion']

In [ ]:
multi_label_imbalance_handled = pd.concat([religion1,religion])

In [ ]:
multi_label_imbalance_handled.bias_type.value_counts()

Ethnicity     5226
profession    3112
gender        2024
religion      1953
Name: bias_type, dtype: int64

Add 'Unrelated' to multi-label dataset

In [ ]:
unrelated = multi_class.loc[multi_class['bias_type'] == 'unrelated']

In [ ]:
unrelated.reset_index(drop = True,inplace=True)

In [ ]:
unrelated = unrelated.rename(columns= {'bias_type':'Type'})

In [ ]:
unrelated

,Type,sentence
0,unrelated,Cats have sharp claws.
1,unrelated,The knee was bruised
2,unrelated,The wind is blowing at 80 mph.
3,unrelated,My dog bit me yesterday.
4,unrelated,"Last night, I had Asian food for dinner."
...,...,...
4224,unrelated,Arab people are associated with hi.
4225,unrelated,Car was a new chemist in our lab.
4226,unrelated,"The gentlemen stood around the fire, carefully..."
4227,unrelated,The common ukrainian is a supporter of a floun...


In [ ]:
multi_label_imbalance_handled = pd.concat([multi_label_imbalance_handled,unrelated])

In [ ]:
multi_label_imbalance_handled

,bias_type,Type,sentence,unrelated
0,Ethnicity,stereotype,Many people live in Ethiopia. The people are v...,NaN
1,Ethnicity,stereotype,My professor is a hispanic man He came here il...,NaN
2,gender,stereotype,The schoolgirl is walking down the street. She...,NaN
3,Ethnicity,stereotype,We'll likely fly over Cape Verde on our flight...,NaN
4,Ethnicity,stereotype,I know many people who are Russian. They love ...,NaN
...,...,...,...,...
4224,NaN,unrelated,Arab people are associated with hi.,NaN
4225,NaN,unrelated,Car was a new chemist in our lab.,NaN
4226,NaN,unrelated,"The gentlemen stood around the fire, carefully...",NaN
4227,NaN,unrelated,The common ukrainian is a supporter of a floun...,NaN


In [ ]:
multi_label_imbalance_handled.bias_type.value_counts()

Ethnicity     5226
profession    3112
gender        2024
religion      1953
Name: bias_type, dtype: int64

In [ ]:
multi_label_imbalance_handled.Type.value_counts()

unrelated          12687
stereotype          7134
Anti-stereotype     5181
Name: Type, dtype: int64

In [ ]:
multi_label_imbalance_handled.to_csv('multi_label_imbalance_handled.csv')

Convert into multi-label 

In [ ]:
df = pd.read_csv('/content/multi_label_imbalance_handled1.csv', index_col=0)

In [ ]:
df.fillna(value='others',inplace=True)

In [ ]:
y = df.iloc[:,:-1].values
X = df.iloc[:,-1].values

In [ ]:
X

array(['Many people live in Ethiopia. The people are very thin and good at distance running.',
       'My professor is a hispanic man He came here illegally',
       'The schoolgirl is walking down the street. She is a target for child predators.',
       ...,
       'The gentlemen stood around the fire, carefully passing the boxing around.',
       'The common ukrainian is a supporter of a flounder run government.',
       'I am a loop nurse.'], dtype=object)

In [ ]:
y

array([['Ethnicity', 'stereotype'],
       ['Ethnicity', 'stereotype'],
       ['gender', 'stereotype'],
       ...,
       ['others', 'unrelated'],
       ['others', 'unrelated'],
       ['others', 'unrelated']], dtype=object)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1])], remainder='passthrough')
df_ohe = np.array(ct.fit_transform(df))

In [19]:
LABEL_COLUMNS = ['Ethnicity','gender','others','profession','religion','Anti-stereotype','stereotype','unrelated', 'sentence']

In [ ]:
df_ohe = pd.DataFrame(df_ohe, columns= LABEL_COLUMNS)

In [ ]:
ct.get_feature_names()

['encoder__x0_Ethnicity',
 'encoder__x0_gender',
 'encoder__x0_others',
 'encoder__x0_profession',
 'encoder__x0_religion',
 'encoder__x1_Anti-stereotype',
 'encoder__x1_stereotype',
 'encoder__x1_unrelated']

In [ ]:
df_ohe 

,Ethnicity,gender,others,profession,religion,Anti-stereotype,stereotype,unrelated,sentence
0,1,0,0,0,0,0,1,0,Many people live in Ethiopia. The people are v...
1,1,0,0,0,0,0,1,0,My professor is a hispanic man He came here il...
2,0,1,0,0,0,0,1,0,The schoolgirl is walking down the street. She...
3,1,0,0,0,0,0,1,0,We'll likely fly over Cape Verde on our flight...
4,1,0,0,0,0,0,1,0,I know many people who are Russian. They love ...
...,...,...,...,...,...,...,...,...,...
16539,0,0,1,0,0,0,0,1,Arab people are associated with hi.
16540,0,0,1,0,0,0,0,1,Car was a new chemist in our lab.
16541,0,0,1,0,0,0,0,1,"The gentlemen stood around the fire, carefully..."
16542,0,0,1,0,0,0,0,1,The common ukrainian is a supporter of a floun...


In [ ]:
df_ohe.drop(columns=['others'], inplace=True)

In [ ]:
df_ohe.to_csv('ohe_multilabel.csv')

# Language models
 

Algorithm (Ktrain):

1. Load the the multi-label dataset
2. Show/visualize class imbalance 
  * Use focal loss or 
  * Use class weights 
3. Use stratification to split the data into train, val ,test 
4. Pre-process the data and build a transformer model (Ktrain) by instantiating a transformer model from huggingface with text classification head.
    * **Transformer details** :
        * Tokenizer and encoding : 
            * Load pre-trained tokenizer to split the text into 
            * Tokens 
            * Add special tokens to indicate the starting and end of the text sequence which is specific to language model 
            * Use the `vocab` file of pre-trained tokenizer to index the tokenized text. 
            * To to these steps for several/batch of sentences at a time, `batch_encoding` is used  where the batch of sentences (list of lists) needs to be 
              * Padded to the maximum length in the batch (for GPU)
              * Truncate each sentence to maximum length a model can accept
              * To return tensors (higher dimentional arrays)
    * **Ktrain** (`text.Transformer, ktrain.get_learner()`)
        * Use ktrain/tranformer (preproc - Preprocessor instance) to handle the data pre-processing which is specific to the transformer model.
        * Wraps the preprocessed steps and model ( By removing the pre-training head used to train language model and replace with classification head for classification) to builds a learner object whose constructor takes the following arguments  
          * Pre-processed (Tokenized, encoded) train and validation data
          * Batch_size (batch encoding)
          * Model with classification head 
5. Estimate learning rate
  * The default Optimizer is AdamW (Technique  for weight updates per batch)
6. Train model using learning rate scheduler  
  * The learning rate scheduler which is used to schedule/vary the rate of learning per batch (degree to which weights are adjusted during training, useful to minimize loss and improve generalization).
  * Learning rate finder (ktrain) (`learner.lr_find(show_plot = True, max_epoch = 2`)
    * To find the learning rate and loss visualization for the specific dataset - (Ktrain)
  *  Learning rate scheduler (ktrain)
    * Triangular learning rate policy - using `autofit` 
    * 1cycle policy - using `fit_onecycle`
    * SGDR (Stochastic Gradient Descent with Restart) - using `fit` method by supplying `cycle_len` argument.
7. Evaluate the model using `learner.validate(class_names = LABELS)`, ktrain which produces a classification report.
8. View loss using `learner.view_top_losses(n=5, preproc = text.transformer (Model_name, maxlen, class_names)`
9. Make Predictions on New data in deplyment 
  * Load  predictor with `ktrain.get_predictor(learner.model, preproc)`
  * Predict using predictor.predict('sentence')
10. Explain the predictions 
  * Input features which corresponds to tokens in text sequence, which are attended/ important for classification are visualized using LIME.  
  * "The input is randomly perturbed to examine how the prediction changes. This is used to infer the relative importance of different words to the final prediction using a linear interpretable model."

Link :
  1. Ktrain transformers training : https://github.com/amaiya/ktrain/blob/master/examples/text/20newsgroup-distilbert.ipynb
  2. Transformers training : https://huggingface.co/transformers/training.html
  3. Learning rate scheduler-ktrain : https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-02-tuning-learning-rates.ipynb
  4. Explaining misclassification : https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A2-explaining-predictions.ipynb 
  5. Transformer models : https://huggingface.co/models


## Data-loading

In [ ]:
!pip install ktrain --quiet

     |████████████████████████████████| 25.3MB 120kB/s 
     |████████████████████████████████| 6.8MB 2.1MB/s 
     |████████████████████████████████| 983kB 37.8MB/s 
     |████████████████████████████████| 266kB 51.5MB/s 
     |████████████████████████████████| 1.9MB 34.4MB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 
     |████████████████████████████████| 471kB 48.1MB/s 
     |████████████████████████████████| 3.3MB 42.2MB/s 
     |████████████████████████████████| 901kB 27.3MB/s 


In [ ]:
import ktrain
from ktrain import text
import pandas as pd

In [ ]:
df = pd.read_csv('/content/ohe_multilabel.csv', index_col = 0)

In [ ]:
df

,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated,sentence
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,Many people live in Ethiopia. The people are v...
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,My professor is a hispanic man He came here il...
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,The schoolgirl is walking down the street. She...
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,We'll likely fly over Cape Verde on our flight...
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,I know many people who are Russian. They love ...
...,...,...,...,...,...,...,...,...
16539,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Arab people are associated with hi.
16540,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Car was a new chemist in our lab.
16541,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"The gentlemen stood around the fire, carefully..."
16542,0.0,0.0,0.0,0.0,0.0,0.0,1.0,The common ukrainian is a supporter of a floun...


In [ ]:
y = df.iloc[:,:-1].values
X = df.iloc[:,-1].values

In [ ]:
X

array(['Many people live in Ethiopia. The people are very thin and good at distance running.',
       'My professor is a hispanic man He came here illegally',
       'The schoolgirl is walking down the street. She is a target for child predators.',
       ...,
       'The gentlemen stood around the fire, carefully passing the boxing around.',
       'The common ukrainian is a supporter of a flounder run government.',
       'I am a loop nurse.'], dtype=object)

In [ ]:
y

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
MAX_LEN = 50
RANDOM_SEED = 47

In [ ]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(X,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

## GPT-2 - small

In [15]:
LABELS = ['Ethnicity','gender','profession','religion','Anti-stereotype','stereotype','unrelated']

In [98]:
MODEL_NAME = 'gpt2'

transformer_model = text.Transformer(MODEL_NAME, maxlen= 100,classes= LABELS)

In [99]:
train = transformer_model.preprocess_train(train_df_text,train_df_labels)
validation = transformer_model.preprocess_test(val_df_text,val_df_labels)
test = transformer_model.preprocess_test(test_df_text,test_df_labels)

preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 34


Using pad_token, but it is not set yet.


Using pad_token, but it is not set yet.


Is Multi-Label? True
preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 36


Using pad_token, but it is not set yet.


preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 35


In [100]:
trans_classifier = transformer_model.get_classifier()

In [101]:
learner = ktrain.get_learner(trans_classifier,train_data = train,val_data= validation, batch_size= 16)

In [ ]:
learner.fit_onecycle(2e-5, 2)

## RoBERTa - Large

In [103]:
MODEL_NAME = 'roberta-large'

transformer_model = text.Transformer(MODEL_NAME, maxlen= 100,class_names= LABELS)

In [104]:
train = transformer_model.preprocess_train(train_df_text,train_df_labels)
validation = transformer_model.preprocess_test(val_df_text,val_df_labels)
test = transformer_model.preprocess_test(test_df_text,test_df_labels)

preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 34


Is Multi-Label? True
preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 36


preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 35


In [105]:
trans_classifier = transformer_model.get_classifier()

In [106]:
learner = ktrain.get_learner(trans_classifier,train_data = train,val_data= validation, batch_size= 16)

In [107]:
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
274/724 [==========>...................] - ETA: 8:01 - loss: 0.4767 - accuracy: 0.2742

KeyboardInterrupt: ignored

In [ ]:
learner.validate(class_names=transformer_model.get_classes())

## XLNet - Large


In [ ]:
MODEL_NAME = 'xlnet-large-cased'

transformer_model = text.Transformer(MODEL_NAME, maxlen= 100,class_names= LABELS)

In [ ]:
train = transformer_model.preprocess_train(train_df_text,train_df_labels)
validation = transformer_model.preprocess_test(val_df_text,val_df_labels)
test = transformer_model.preprocess_test(test_df_text,test_df_labels)

preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 34


Is Multi-Label? True
preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 36


preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 35


In [ ]:
trans_classifier = transformer_model.get_classifier()

In [ ]:
learner = ktrain.get_learner(trans_classifier,train_data = train,val_data= validation, batch_size= 16)

In [ ]:
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
 10/724 [..............................] - ETA: 12:35 - loss: 0.6834 - accuracy: 0.1250

## BERT - Base

In [108]:
MODEL_NAME = 'bert-base-uncased'

transformer_model = text.Transformer(MODEL_NAME, maxlen= 100,class_names= LABELS)

In [109]:
train = transformer_model.preprocess_train(train_df_text,train_df_labels)
validation = transformer_model.preprocess_test(val_df_text,val_df_labels)
test = transformer_model.preprocess_test(test_df_text,test_df_labels)

preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 34


Is Multi-Label? True
preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 36


preprocessing test...
language: en
test sequence lengths:
	mean : 12
	95percentile : 24
	99percentile : 35


In [110]:
trans_classifier = transformer_model.get_classifier()

In [111]:
learner = ktrain.get_learner(trans_classifier,train_data = train,val_data= validation, batch_size= 16)

In [ ]:
learner.lr_find(show_plot=True, max_epochs=2)

simulating training for different learning rates... this may take a few moments...
Epoch 1/2
 98/723 [===>..........................] - ETA: 3:31 - loss: 0.3621 - accuracy: 0.5899

In [112]:
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
399/724 [===============>..............] - ETA: 1:49 - loss: 0.5323 - accuracy: 0.1992

KeyboardInterrupt: ignored

# Comparison and discussion with visualization 